<a href="https://colab.research.google.com/github/tousif47/Mini-LLM/blob/main/Scratch_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import pandas as pd
import re

from keras.utils import to_categorical
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

print("Done")

Done


In [47]:
data_text = open("/content/drive/MyDrive/Practice Playing with code/health.txt", "r")

print("Number of characters in text file :", len(data_text.read()))

Number of characters in text file : 7126


In [64]:
# Preprocessing the text data
# Basic text preprocessing because a simple dataset with minimal noise has been selected to simplify the code
# To maintain uniformity all words has been converted to lower case and any words with length less than 3 has been removed
# Punctuations are also removed

def text_preprocess(text):

    # Lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)

    # Remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString)
    long_words = []

    # Remove short words
    for i in newString.split():
        if len(i) >= 3:
            long_words.append(i)
    return (" ".join(long_words)).strip()

# Calling the preprocess function
data_text = open("/content/drive/MyDrive/Practice Playing with code/health.txt", "r")
data_preprocessed = text_preprocess(data_text.read())

print(data_preprocessed[:100])

being healthy and fit simple terms means taking good care the body should remember that healthy mind


In [71]:
# Creating sequences
# Taking 40 characters as context, model will try to predict the next character

def sequencer(text):
    length = 40
    sequences = list()

    for i in range(length, len(text)):

        # Select sequence of tokens
        seq = text[i-length:i+1]

        # Store
        sequences.append(seq)

    print("Total Sequences: %d" % len(sequences))
    return sequences

# Calling the sequencer
sequences = sequencer(data_preprocessed)
print("\n", sequences[0], "\n", sequences[1], "\n", sequences[2], "\n", sequences[3], "\n", sequences[4])

Total Sequences: 6322

 being healthy and fit simple terms means  
 eing healthy and fit simple terms means t 
 ing healthy and fit simple terms means ta 
 ng healthy and fit simple terms means tak 
 g healthy and fit simple terms means taki
